In [1]:
import os
import pandas as pd
import numpy as np
from PIL import Image

In [2]:
images_path = "Datasets\MIMIC\matched_images_and_masks_mimic_224\images"
mask_path = "Datasets\MIMIC\matched_images_and_masks_mimic_224\masks"
image_type = ".jpg"

def plot_one_image(image_id):
    full_path = f"{images_path}/{image_id}{image_type}"
    if os.path.exists(full_path):
        with Image.open(full_path) as im:
                im = im.convert("RGB")
                return im

def plot_one_mask(mask_id):
    full_path = f"{mask_path}/{mask_id}{image_type}"
    if os.path.exists(full_path):
        with Image.open(full_path) as im:
                im = im.convert("RGB")
                return im
    return None

In [3]:
# LOAD first image found on images path
first_image_id = os.listdir(images_path)[0].split(".")[0]
first_mask_id = os.listdir(mask_path)[0].split(".")[0]
print(f"First image ID: {first_image_id}")
image = plot_one_image(first_image_id)
print(f"First mask ID: {first_mask_id}")
mask = plot_one_mask(first_mask_id)

First image ID: 000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d
First mask ID: 000046e4-e4d7f796-72c3dba4-8b67a485-0eea211d-mask


In [4]:
mask.size, image.size

(224, 224)

(224, 224)

In [6]:
from utils.gpt_models import DINOEncoder
encoder = DINOEncoder()

In [12]:
patches.shape

torch.Size([1, 196, 384])

In [14]:
from torchvision import transforms
from utils.temp_utils import dino_image_transform
IMG_SIZE = 224
tf = dino_image_transform(img_size=IMG_SIZE)
image_tensor = tf(image)
cls, patches =encoder.forward(pixel_values=image_tensor.unsqueeze(0))

((224, 224), torch.Size([1, 14, 14, 384]))